# Boosting

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import CleaningCars as cc

from sklearn.metrics import mean_squared_error

In [2]:
df=pd.read_csv('vehicles.csv')

In [3]:
df=cc.ultimateClean(df)

Remove outliers...
Remove outliers...
Cleaned outliers !
Imputed Missing Values
One hot encodings done!
Total Time:  7.21435885032018  minutes


In [4]:
from sklearn.preprocessing import StandardScaler

#Standardise features
cols_to_norm = ['year','odometer','lat','long']

df[cols_to_norm] = StandardScaler().fit_transform(df[cols_to_norm])

In [5]:
X=df.drop("price",axis=1)
y=df['price']

#data_matrix=xgb.DMatrix(data=X, label=y)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# XGboost

In [17]:
import xgboost as xgb

xgb_reg = xgb.XGBRegressor(learning_rate = 0.1, 
                           max_depth = 20, 
                           alpha = 10, 
                           n_estimators = 200)

xgb_reg.fit(X_train,y_train)
preds_train= xgb_reg.predict(X_train)
preds_test = xgb_reg.predict(X_test)


print(f"MSE on train set: {mean_squared_error(y_train, preds_train)}")
print(f"RMSE on train set: {mean_squared_error(y_train, preds_train)**0.5}")
print()
print(f"MSE on test set:  {mean_squared_error(y_test, preds_test)}")
print(f"RMSE on test set:  {mean_squared_error(y_test, preds_test)**0.5}")

MSE on train set: 2838661.113233323
RMSE on train set: 1684.832666241168

MSE on test set:  19441137.871776145
RMSE on test set:  4409.2105724013845


# LGBM

In [45]:
import lightgbm as lgb
train_data=lgb.Dataset(X_train,y_train)



lgbm_params={"learning_rate":0.1,
             "boosting_type":"gbdt",  
             "objective":"regression",
             "metric":"mse",
             "num_leaves":2000,
             "max_depth":40}

model=lgb.train(lgbm_params, train_data, 1000) 
preds_train=model.predict(X_train)
preds_test=model.predict(X_test)


print(f"MSE on train set: {mean_squared_error(y_train, preds_train)}")
print(f"RMSE on train set: {mean_squared_error(y_train, preds_train)**0.5}")
print()
print(f"MSE on test set:  {mean_squared_error(y_test, preds_test)}")
print(f"RMSE on test set:  {mean_squared_error(y_test, preds_test)**0.5}")

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022029 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1053
[LightGBM] [Info] Number of data points in the train set: 310668, number of used features: 100
[LightGBM] [Info] Start training from score 14235.128269
MSE on train set: 1450707.5653418957
RMSE on train set: 1204.4532225627925

MSE on test set:  17791916.687488742
RMSE on test set:  4218.046548757937
